In [ ]:
!pip install transformers datasets torch scikit-learn

In [1]:
import pandas as pd

df = pd.read_csv("C:\\Users\\dell\\PycharmProjects\\Day6-InvoiceClassification\\Company.csv")

df

,text,label,word_count
0,order id 10718 shipping details ship name k...,ShippingOrder,120
1,invoice order id 10707 customer id arout ord...,invoice,66
2,order id 10448 shipping details ship name r...,ShippingOrder,96
3,invoice order id 11068 customer id queen ord...,invoice,68
4,order id 10656 shipping details ship name g...,ShippingOrder,109
...,...,...,...
2671,order id 10326 shipping details ship name b...,ShippingOrder,111
2672,purchase orders order id order date customer n...,purchase Order,39
2673,invoice order id 10460 customer id folko ord...,invoice,59
2674,stock report for 2018-01 category meat poult...,report,46


In [2]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re

nltk.download("stopwords")

stop_words = set(stopwords.words("english"))
stemmer = PorterStemmer()

def clean_text(text):
    text = text.lower()
    text = re.sub(r"[^\w\s]", "", text)
    text = re.sub(r"\s+", " ", text)
    text = " ".join([stemmer.stem(word) for word in text.split() if word not in stop_words])
    return text

df["Cleaned Text"] = df["text"].apply(clean_text)

df


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,text,label,word_count,Cleaned Text
0,order id 10718 shipping details ship name k...,ShippingOrder,120,order id 10718 ship detail ship name königlich...
1,invoice order id 10707 customer id arout ord...,invoice,66,invoic order id 10707 custom id arout order da...
2,order id 10448 shipping details ship name r...,ShippingOrder,96,order id 10448 ship detail ship name rancho gr...
3,invoice order id 11068 customer id queen ord...,invoice,68,invoic order id 11068 custom id queen order da...
4,order id 10656 shipping details ship name g...,ShippingOrder,109,order id 10656 ship detail ship name great lak...
...,...,...,...,...
2671,order id 10326 shipping details ship name b...,ShippingOrder,111,order id 10326 ship detail ship name bólido co...
2672,purchase orders order id order date customer n...,purchase Order,39,purchas order order id order date custom name ...
2673,invoice order id 10460 customer id folko ord...,invoice,59,invoic order id 10460 custom id folko order da...
2674,stock report for 2018-01 category meat poult...,report,46,stock report 201801 categori meat poultri id c...


In [3]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

In [4]:
df

,text,label,word_count,Cleaned Text
0,order id 10718 shipping details ship name k...,0,120,order id 10718 ship detail ship name königlich...
1,invoice order id 10707 customer id arout ord...,1,66,invoic order id 10707 custom id arout order da...
2,order id 10448 shipping details ship name r...,0,96,order id 10448 ship detail ship name rancho gr...
3,invoice order id 11068 customer id queen ord...,1,68,invoic order id 11068 custom id queen order da...
4,order id 10656 shipping details ship name g...,0,109,order id 10656 ship detail ship name great lak...
...,...,...,...,...
2671,order id 10326 shipping details ship name b...,0,111,order id 10326 ship detail ship name bólido co...
2672,purchase orders order id order date customer n...,2,39,purchas order order id order date custom name ...
2673,invoice order id 10460 customer id folko ord...,1,59,invoic order id 10460 custom id folko order da...
2674,stock report for 2018-01 category meat poult...,3,46,stock report 201801 categori meat poultri id c...


In [5]:
from datasets import Dataset

dataset = Dataset.from_pandas(df[["Cleaned Text", "label"]])

In [6]:
dataset

Dataset({
    features: ['Cleaned Text', 'label'],
    num_rows: 2676
})

In [7]:
split_dataset = dataset.train_test_split(test_size = 0.2, seed = 42)

train_texts = split_dataset['train']
test_texts = split_dataset['test']

In [8]:
from transformers import AutoTokenizer

In [9]:
model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples["Cleaned Text"], padding='max_length', truncation=True, max_length = 128)

train_dataset = train_texts.map(tokenize_function, batched = True)
test_dataset = test_texts.map(tokenize_function, batched = True)

train_dataset.set_format(type = 'torch', columns = ['input_ids','attention_mask', 'label'])
test_dataset.set_format(type = 'torch', columns = ['input_ids','attention_mask', 'label'])

Map:   0%|          | 0/2140 [00:00<?, ? examples/s]

Map:   0%|          | 0/536 [00:00<?, ? examples/s]

In [10]:
train_dataset[0]

{'label': tensor(2),
 'input_ids': tensor([  101, 16405, 11140,  3022,  2344,  2344,  8909,  2344,  3058,  7661,
          2171,  8746, 11387,  2418,  2692, 20958,  2683,  6285,  2050, 15214,
         21007,  5054,  4031,  4031,  8909,  4031, 24110,  3775,  3775,  3131,
          3976,  2484, 19739, 20486,  2050, 10392,  2050,  1022,  1018,  1019,
          5187,  9300,  2627,  2072,  1019,  3590,  1022,  3931,  1015,   102,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,   

In [11]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Load pre-trained BERT model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label_encoder.classes_))

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="weighted")
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\dell\AppData\Local\Temp\ipykernel_4268\3051607116.py:26: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
C:\Users\dell\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.005296,1.000000,1.000000,1.000000,1.000000
2,No log,0.002355,1.000000,1.000000,1.000000,1.000000
3,No log,0.001932,1.000000,1.000000,1.000000,1.000000


C:\Users\dell\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\Users\dell\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=402, training_loss=0.06508416322926383, metrics={'train_runtime': 5373.2552, 'train_samples_per_second': 1.195, 'train_steps_per_second': 0.075, 'total_flos': 422300827054080.0, 'train_loss': 0.06508416322926383, 'epoch': 3.0})

In [13]:
import pickle

model_save_path = './model/'
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

('./model/tokenizer_config.json',
 './model/special_tokens_map.json',
 './model/vocab.txt',
 './model/added_tokens.json',
 './model/tokenizer.json')

In [16]:
import pickle

label_encoder_path = './model/label_encoder.pkl'  # Add a filename
with open(label_encoder_path, 'wb') as f:
    pickle.dump(label_encoder, f)

print("Model, tokenizer, and label encoder saved")

Model, tokenizer, and label encoder saved


In [18]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import numpy as np

# Load fine-tuned model and tokenizer
model_name = "./model/"  # Update with your saved model path
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Function for prediction
def predict(text):
    # Clean the input text (if you have a cleaning function)
    text = clean_text(text)

    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
    
        # Make prediction
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_class = torch.argmax(logits, dim=1).item()

    # Convert scalar to array-like and return inverse transform
    return label_encoder.inverse_transform([predicted_class])[0]  # Return the predicted class value

# Example usage
input_text = "invoice order id 10707 customer id arout order date 2017-10-16 customer detail"
print(predict(input_text))



invoice
